In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import warnings
warnings.filterwarnings('ignore')


from revolve.grids import MLPParameterGrid
from revolve.architectures import MLPStrategy
from revolve.algorithms import EvolutionaryAlgorithmElitism
from revolve.operators import (
    Operations,
    random_mutation,
    uniform_crossover,
    roulette_wheel_selection,
)

In [ ]:
train, test = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)
x_train_full, y_train_full = train
x_test, y_test = test

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, test_size=0.1, random_state=42)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

In [ ]:
operations = Operations()
operations.register(roulette_wheel_selection)
operations.register(uniform_crossover, probability=0.5)
operations.register(random_mutation, probability=0.1)

In [ ]:
operations.get_operations()

In [ ]:
mlp_params = MLPParameterGrid(
    input_shape=(x_train.shape[1],),
    regression_target=1,
    regression_activation= "relu",
    hidden_neurons = [0, 16, 32, 64, 128, 258, 512, 1024, 2048],
    activation=['relu', 'swish'],
    dropout=[0.1, 0.2, 0.3, 0.4, 0.5],
    batch_size=[16, 32, 64],
    optimizer=['adam', 'sgd', 'rmsprop'],
    learning_rate=[7.5e-3, 5.0e-3, 2.5e-3, 1.0e-4, 7.5e-4, 5e-4, 2.5e-4],
    l1=[1e-5, 1e-6],
    l2=[1e-5, 1e-6],
)

In [ ]:
mlp_params.static_parameters

In [ ]:
mlp_params.learnable_parameters

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=2)

mlp = MLPStrategy(
    max_fc = 3,
    squeeze_fc=False,
    epochs=50,
    callback=callback,
    loss='mean_squared_error',
    metric = tfa.metrics.RSquare(),
    parameters = mlp_params
)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))

data = (train_data, valid_data, test_data)

In [ ]:
ea = EvolutionaryAlgorithmElitism(
    strategy=mlp,
    pop_size=50,
    elitism_size=1,
    operations=operations,
)

In [ ]:
best_chromosome = ea.fit(
    data,
    generations=20,
)

In [ ]:
df = ea.results_df()

In [ ]:
df.head()

In [ ]:
model = best_chromosome.decode(mlp_params)
model.summary()

In [ ]:
best_chromosome_row = df[df.loss == df.loss.min()]

In [ ]:
dict(best_chromosome_row.learnt_parameters)

In [ ]:
dict(best_chromosome_row.static_parameters)

In [19]:
dict(best_chromosome_row.static_parameters)

{531: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 600: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 750: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 800: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 850: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 900: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'},
 950: {'input_shape': (13,),
  'regression_target': 1,
  'regression_activation': 'relu'}}

In [20]:
elite_models = ea.elite_models

In [21]:
elite_models